# Develop a Custom Object Detector - RetinaNet Version

###### Let's Upgrade Anaconda if you have it installed

In [0]:
# Enable to update/upgrade anaconda
#!conda update conda
#!conda upgrade anaconda

###### Next we will be doing some neccesary imports

In [0]:
import os
import fnmatch
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

from xml.etree import ElementTree as et
from urllib.request import urlretrieve
from google.colab import drive
from google.colab import files



%matplotlib inline

###### Let's link our drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###### Now install the required packaget to continue the flow

In [4]:
# Enable if you did not installed the packages
%cd /content/drive/My\ Drive/ObjectDetector
!pip install -r requirements.txt

/content/drive/My Drive/ObjectDetector
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


###### Let's mount our drive and create the work folder

In [5]:
!mkdir /content/drive/My\ Drive/ObjectDetector
%cd /content/drive/My\ Drive/ObjectDetector
!unzip Dataset.zip &> /dev/null

mkdir: cannot create directory ‘/content/drive/My Drive/ObjectDetector’: File exists
/content/drive/My Drive/ObjectDetector


In [0]:
xml_train_path  = os.path.join('Dataset','Train')
xml_train_files = fnmatch.filter(os.listdir(xml_train_path), "*.xml")
xml_train_files_full_path = [os.path.join(xml_train_path, p) for p in xml_train_files]

xml_test_path   = os.path.join('Dataset', 'Test')
xml_test_files  = fnmatch.filter(os.listdir(xml_test_path), "*.xml")
xml_test_files_full_path = [os.path.join(xml_test_path, p) for p in xml_test_files]

###### see a dataset xml file path

In [7]:
xml_test_files_full_path[:5]

['Dataset/Test/0710_nc_red_fox_jpg.xml',
 'Dataset/Test/0_1024px-Black_fox.xml',
 'Dataset/Test/0_badger1_result.xml',
 'Dataset/Test/0_Eurasian-badger-Meles-meles-emerging-from-sett-England.xml',
 'Dataset/Test/1 (1).xml']

In [8]:
xml_train_files_full_path[:5]

['Dataset/Train/asian-badger.xml',
 'Dataset/Train/baby-badger-drinking-figurine.xml',
 'Dataset/Train/baby-fox.xml',
 'Dataset/Train/badger (1).xml',
 'Dataset/Train/badger (10).xml']

###### see how many images are in the train and test

In [9]:
ntrain = len(xml_train_files)
ntest  = len(xml_test_files)
print(f'There are {ntrain} images for training.\nThera are {ntest} images for validation.')

There are 236 images for training.
Thera are 60 images for validation.


###### test xml extraction for train and test data

In [0]:
tree = et.parse(xml_train_files_full_path[0])
root = tree.getroot()

In [11]:
print(et.tostring(root, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<annotation>
	<folder>badger_resized</folder>
	<filename>asian-badger.jpg</filename>
	<path>Dataset\Train\asian-badger.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>720</width>
		<height>514</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>badger</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>14</xmin>
			<ymin>1</ymin>
			<xmax>711</xmax>
			<ymax>450</ymax>
		</bndbox>
	</object>
</annotation>


In [12]:
for fname in root.iter('filename'):
    print(fname.text)

asian-badger.jpg


In [13]:
for path in root.iter('path'):
    print(path.text)

Dataset\Train\asian-badger.jpg


In [14]:
for s in root.iter('size'):
    w = int(s.find('width').text)
    h = int(s.find('height').text)
    print(w, h)

720 514


In [15]:
for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    print(xmin, ymin, xmax, ymax)

14 1 711 450


In [16]:
for class_ in root.findall('.//object'):
    name = class_.find('name').text
    print(name)

badger


###### treat xml data

In [0]:
# Accomodate the path
for xmlpath in xml_train_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for fname in root.iter('filename'):
        name = fname.text
    for path in root.iter('path'):
        path.text = os.path.join(xml_train_path, name)
    tree.write(xmlpath)

In [0]:
# Accomodate the path
for xmlpath in xml_test_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for fname in root.iter('filename'):
        name = fname.text
    for path in root.iter('path'):
        path.text = os.path.join(xml_test_path, name)
    tree.write(xmlpath)

In [19]:
for path in root.iter('path'):
    print(path.text)

Dataset/Test/_fox_16x9.jpg


###### Make an array to feed later a dataframe

In [0]:
values_train = []
for xmlpath in xml_train_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for path in root.iter('path'):
        imagepath = path.text
    for b in root.findall('.//bndbox'):
        xmin = int(b.find('xmin').text)
        ymin = int(b.find('ymin').text)
        xmax = int(b.find('xmax').text)
        ymax = int(b.find('ymax').text)
    for class_ in root.findall('.//object'):
        class_name = class_.find('name').text
    values_train.append([imagepath, xmin, ymin, xmax, ymax, class_name])

In [21]:
values_train[0], values_train[1], values_train[2], values_train[3], values_train[4]

(['Dataset/Train/asian-badger.jpg', 14, 1, 711, 450, 'badger'],
 ['Dataset/Train/baby-badger-drinking-figurine.jpg',
  16,
  72,
  720,
  436,
  'badger'],
 ['Dataset/Train/baby-fox.jpg', 187, 76, 642, 425, 'fox'],
 ['Dataset/Train/badger (1).jpg', 7, 30, 628, 485, 'badger'],
 ['Dataset/Train/badger (10).jpg', 63, 22, 720, 478, 'badger'])

In [0]:
values_test = []
for xmlpath in xml_test_files_full_path:
    tree = et.parse(xmlpath)
    root = tree.getroot()
    for path in root.iter('path'):
        imagepath = path.text
    for b in root.findall('.//bndbox'):
        xmin = int(b.find('xmin').text)
        ymin = int(b.find('ymin').text)
        xmax = int(b.find('xmax').text)
        ymax = int(b.find('ymax').text)
    for class_ in root.findall('.//object'):
        class_name = class_.find('name').text
    values_test.append([imagepath, xmin, ymin, xmax, ymax, class_name])

In [23]:
values_test[0], values_test[1], values_test[2], values_test[3], values_test[4]

(['Dataset/Test/0710_nc_red_fox_jpg.jpg', 9, 59, 646, 374, 'fox'],
 ['Dataset/Test/0_1024px-Black_fox.jpg', 31, 52, 666, 423, 'fox'],
 ['Dataset/Test/0_badger1_result.jpg', 42, 100, 582, 456, 'badger'],
 ['Dataset/Test/0_Eurasian-badger-Meles-meles-emerging-from-sett-England.jpg',
  274,
  119,
  602,
  508,
  'badger'],
 ['Dataset/Test/1 (1).jpg', 236, 6, 658, 261, 'badger'])

###### make dataframes and write to csv

In [24]:
train_df = pd.DataFrame(values_train)
train_df.head()

,0,1,2,3,4,5
0,Dataset/Train/asian-badger.jpg,14,1,711,450,badger
1,Dataset/Train/baby-badger-drinking-figurine.jpg,16,72,720,436,badger
2,Dataset/Train/baby-fox.jpg,187,76,642,425,fox
3,Dataset/Train/badger (1).jpg,7,30,628,485,badger
4,Dataset/Train/badger (10).jpg,63,22,720,478,badger


In [25]:
test_df = pd.DataFrame(values_test)
test_df.head()

,0,1,2,3,4,5
0,Dataset/Test/0710_nc_red_fox_jpg.jpg,9,59,646,374,fox
1,Dataset/Test/0_1024px-Black_fox.jpg,31,52,666,423,fox
2,Dataset/Test/0_badger1_result.jpg,42,100,582,456,badger
3,Dataset/Test/0_Eurasian-badger-Meles-meles-eme...,274,119,602,508,badger
4,Dataset/Test/1 (1).jpg,236,6,658,261,badger


In [26]:
column_names = ['image_path','xmin','ymin', 'xmax','ymax','class_name']
train_df.columns = column_names
test_df.columns = column_names
test_df.head()

,image_path,xmin,ymin,xmax,ymax,class_name
0,Dataset/Test/0710_nc_red_fox_jpg.jpg,9,59,646,374,fox
1,Dataset/Test/0_1024px-Black_fox.jpg,31,52,666,423,fox
2,Dataset/Test/0_badger1_result.jpg,42,100,582,456,badger
3,Dataset/Test/0_Eurasian-badger-Meles-meles-eme...,274,119,602,508,badger
4,Dataset/Test/1 (1).jpg,236,6,658,261,badger


In [0]:
TRAIN_ANNOTATIONS = 'train_annotations.csv'
TEST_ANNOTATIONS  = 'test_annotations.csv' 
CLASSES           = 'classes.csv'

class_names = ['fox', 'badger']

with open(CLASSES, 'w') as classes:
    for i, cl in enumerate(class_names):
        classes.write(f'{cl}, {i}\n')


train_df.to_csv(TRAIN_ANNOTATIONS, index=False, header=False)
test_df.to_csv(TEST_ANNOTATIONS, index=False, header=False)

### Download the Repo and Weights

In [28]:
BASE_MODEL_PATH = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
BASE_MODEL_NAME = 'resnet50_coco_best_v2.1.0.h5'

urlretrieve(BASE_MODEL_PATH, BASE_MODEL_NAME)

('resnet50_coco_best_v2.1.0.h5', <http.client.HTTPMessage at 0x7fd243328b00>)

### Compile RetinaNet

In [34]:
%cd /contet/drive/My\ Drive/ObjectDetector/
!git clone https://github.com/fizyr/keras-retinanet
%cd keras-retinanet
!git checkout 42068ef9e406602d92a1afe2ee7d470f7e9860df
!python setup.py install


#%cd /contet/drive/My\ Drive/ObjectDetector/keras-retinanet
#!pip install numpy --user
#!pip install . --user
#!python setup.py build_ext --inplace
#%cd /contet/drive/My\ Drive/ObjectDetector/

[Errno 2] No such file or directory: '/contet/drive/My Drive/ObjectDetector/'
/content/drive/My Drive/ObjectDetector
fatal: destination path 'keras-retinanet' already exists and is not an empty directory.
/content/drive/My Drive/ObjectDetector/keras-retinanet
error: Your local changes to the following files would be overwritten by checkout:
	keras_retinanet/bin/convert_model.py
	keras_retinanet/bin/debug.py
	keras_retinanet/bin/evaluate.py
	keras_retinanet/bin/train.py
Please commit your changes or stash them before you switch branches.
Aborting
running install
running bdist_egg
running egg_info
creating keras_retinanet.egg-info
writing keras_retinanet.egg-info/PKG-INFO
writing dependency_links to keras_retinanet.egg-info/dependency_links.txt
writing entry points to keras_retinanet.egg-info/entry_points.txt
writing requirements to keras_retinanet.egg-info/requires.txt
writing top-level names to keras_retinanet.egg-info/top_level.txt
writing manifest file 'keras_retinanet.egg-info/SOURC

###### Do a directory for leave the partial and total training weights

In [30]:
!pip install --user --upgrade keras-retinanet

Requirement already up-to-date: keras-retinanet in /root/.local/lib/python3.6/site-packages (0.5.1)


In [35]:
%cd /content/drive/My Drive/ObjectDetector
!mkdir snapshot

/content/drive/My Drive/ObjectDetector
mkdir: cannot create directory ‘snapshot’: File exists


### Start Training!

In [0]:
BATCH_SIZE = 4                           # 4 images per batch
STEPS      = str(int(ntrain/BATCH_SIZE)) # 236/4 = 59 steps, str to pass an ascii to command line
EPOCHS     = str(10)                     # 10 rounds of the whole image dataset, str to pass an ascii to command line
WEIGHTS    = BASE_MODEL_NAME             # same as <current_working_directory>/resnet50_coco_best_v2.1.0.h5
SNAPSHOT   = 'snapshot'                  # same as <current_working_directory>/snapshot
TRAIN_BASE = TRAIN_ANNOTATIONS           # same as <current_working_directory>/train_annotations.csv
TEST_BASE  = TEST_ANNOTATIONS            # same as <current_working_directory>/test_annotations.csv
#CLASSES =                              # same as <current_working_directory>/classes.csv
BATCH_SIZE = str(BATCH_SIZE)             # normally is a number, but in command line is an ascii

In [37]:
!retinanet-train \
--batch-size $BATCH_SIZE \
--steps $STEPS --epochs $EPOCHS \
--weights $WEIGHTS \
--snapshot-path $SNAPSHOT \
csv $TRAIN_BASE $CLASSES > train_log.txt   # see the log at <current_working_directory>/log.txt

# This above is the in command line as...
#retinanet-train --batch-size 4 --steps 59 --epochs 10 \
#--weights resnet50_coco_best_v2.1.0.h5 \
#--snapshot-path snapshot \
#csv train_annotations.csv classes.csv

Using TensorFlow backend.
2020-06-16 15:00:34.585700: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:00:36.721629: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-16 15:00:36.788922: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-16 15:00:36.789809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-06-16 15:00:36.789876: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:00:37.050695: I tensorflow/str

###### Convert the model to an output h5 file

In [0]:
NAME             = 'resnet50' + '_csv_' + EPOCHS + '.h5' # the last model name at <current_working_directory>/resnet50_csv_10.h5
INPUT_MODEL_NAME =  os.path.join(SNAPSHOT, NAME)  # same as <current_working_directory>/snapshot/resnet10_csv_10.h5
OUTPUT_MODEL_NAME= 'output.h5'                    # the output name at <current_orking_directory>

In [39]:
!retinanet-convert-model $INPUT_MODEL_NAME $OUTPUT_MODEL_NAME > convert_log.txt # The model conversion

# The above is the same as...
#retinanet-convert-model resnet50_csv_10.h5 output.h5

Using TensorFlow backend.
2020-06-16 15:55:30.151305: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:55:32.142058: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-16 15:55:32.168360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-16 15:55:32.169144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-06-16 15:55:32.169191: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:55:32.183212: I tensorflow/str

### Evaluate The Model

In [40]:
!retinanet-evaluate csv $TEST_BASE $CLASSES $OUTPUT_MODEL_NAME > test_log.txt

# The above is the same as ...
#retinanet-evaluate csv test_annotations.csv classes.csv output.h5

Using TensorFlow backend.
2020-06-16 15:55:49.102233: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:55:51.373205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-16 15:55:51.392146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-16 15:55:51.393038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-06-16 15:55:51.393096: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-16 15:55:51.398721: I tensorflow/str

### Test the Model

In [0]:
def display_output(imagePath):
    src = cv2.imread(imagePath,1)
    output = cv2.imread("prediction.jpg")

    plt.figure(figsize=[20,8])
    plt.subplot(121)
    plt.imshow(src[:,:,::-1])
    plt.title("Original Image")
    plt.subplot(122)
    plt.imshow(output[:,:,::-1])
    plt.title("Predictions")
    plt.show()

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cgi_badger_fox.jpg to cgi_badger_fox.jpg
User uploaded file "cgi_badger_fox.jpg" with length 94673 bytes


In [0]:
test_image_full_path = 'cgi_badger_fox.jpg'
CONFIDENCE = str(0.5)
#labels_to_names = pd.read_csv(CLASSES, header=None).T.loc[0].to_dict()
#labels_to_names

In [0]:
%cd /content/drive/My\ Drive/ObjectDetector/keras-retinanet

!python predict.py --model $OUTPUT_MODEL_NAME \
--labels $CLASSES \
--image $test_image_full_path --confidence $CONFIDENCE
#!python ../predict.py --model ../output.h5 --labels ../classes.csv --image ../cgi_badger_fox.jpg --confidence 0.5

%cd /content/drive/My\ Drive/ObjectDetector/

/content/drive/My Drive/ObjectDetector/keras-retinanet
Using TensorFlow backend.
2020-06-16 08:55:17.368940: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "../predict.py", line 6, in <module>
    from keras_retinanet.models.resnet import custom_objects
ImportError: cannot import name 'custom_objects'
Using TensorFlow backend.
2020-06-16 08:55:20.650585: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "../predict.py", line 6, in <module>
    from keras_retinanet.models.resnet import custom_objects
ImportError: cannot import name 'custom_objects'
/content/drive/My Drive/ObjectDetector


In [0]:
# Explained here
# https://stackoverflow.com/questions/48741213/module-not-found-error-on-google-collaboratory
!pip install --upgrade git+https://github.com/broadinstitute/keras-resnet

  Cloning https://github.com/broadinstitute/keras-resnet to /tmp/pip-req-build-izndl5ub
  Running command git clone -q https://github.com/broadinstitute/keras-resnet /tmp/pip-req-build-izndl5ub
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=22144 sha256=fb5715435408844879351176b9dd120c1da837493dce48cc6e538d7800e2ee9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7q1l2od/wheels/10/52/f3/6a1fdbfb022ce9abfdf00a1ca7e90cef71dea99976edbcb53f
Successfully built keras-resnet
ERROR: keras-retinanet 0.5.1 has requirement keras-resnet==0.1.0, but you'll have keras-resnet 0.2.0 which is incompatible.
  Found existing installation: keras-resnet 0.1.0
    Uninstalling keras-resnet-0.1.0:
      Successfully uninstalled keras-resnet-0.1.0
